In [ ]:
!pip install --upgrade pip

In [ ]:
!pip uninstall -y transformers torch bitsandbytes peft accelerate trl
!pip install -U "torch==2.4.1" "transformers==4.46.3" "accelerate==1.1.1" "bitsandbytes>=0.43.3" "peft==0.13.2" "trl==0.12.1" "hf_transfer"

In [1]:
from huggingface_hub import notebook_login
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig, 
    TrainingArguments, 
    Trainer, 
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [2]:
# Hugging Face 로그인 (필요시 실행)
notebook_login()

In [4]:
# 1. 설정
model_id = 'kakaocorp/kanana-1.5-8b-instruct-2505'
output_dir = "./kanana-customer-analysis"

In [5]:
# 2. 4-bit 양자화 설정 (RTX 3090/4090 공통 최적화)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 # 3090은 bf16 지원
)

# 3. 토크나이저 로드 (Llama-3 스타일)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 4. 모델 로드 (set_submodule 에러 방지를 위해 안정 버전 라이브러리 사용)
print("Loading Kanana Model...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="sdpa" # 3090에서 매우 빠름
)

Loading Kanana Model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
# 5. LoRA 설정
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16, 
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

In [7]:
# 6. 시스템 프롬프트 정의
SYSTEM_PROMPT = """상담 스크립트에서 고객의 성향을 분류하세요.
전체적인 맥락을 참고하되 판단의 근거는 고객의 발화에 한정합니다.

### 분류 규칙
1. 제시된 성향 키워드 중 가장 적절한 한가지만 선택
2. 설명이나 판단 근거는 절대 출력하지말고 오직 하나의 키워드만 출력한다
3. 여러 개의 성향을 가질 경우 S3 > S2 > S1 > N2 > N1 의 순서로 우선 순위를 가진다

### 성향 키워드 목록
- N1: 일반형. 큰 특징이 없고 바로 문의사항을 말함
- N2: 수다형. 사적인 이야기나 본인 상황을 길게 설명함
- S1: 급한성격형. 빠른 처리를 선호함
- S2: 디지털미아형. 기술적인 조작에 서툴고 어플 사용을 어려워함
- S3: 불만형. 분노, 짜증을 드러냄"""

In [8]:
def tokenize_function(examples):
    tokenized_inputs = {"input_ids": [], "labels": [], "attention_mask": []}
    for script, output in zip(examples['script'], examples['truth']):
        # Kanana/Llama-3 공식 포맷
        prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{SYSTEM_PROMPT}<|eot_id|>"
        prompt += f"<|start_header_id|>user<|end_header_id|>\n\n{script}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        answer = f"{output}<|eot_id|>"
        
        p_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]
        a_ids = tokenizer(answer, add_special_tokens=False)["input_ids"]
        
        input_ids = (p_ids + a_ids)[:1024]
        labels = ([-100] * len(p_ids) + a_ids)[:1024]
        
        # 패딩
        padding_len = 1024 - len(input_ids)
        tokenized_inputs["input_ids"].append(input_ids + [tokenizer.pad_token_id] * padding_len)
        tokenized_inputs["labels"].append(labels + [-100] * padding_len)
        tokenized_inputs["attention_mask"].append([1] * len(input_ids) + [0] * padding_len)
    return tokenized_inputs

In [9]:
df = pd.read_csv("train.csv")
dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [10]:
# 7. 학습 설정 (RTX 3090용 성능 최적화)
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,        # 3090이면 4까지 넉넉합니다
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,                           # 3090 필수 옵션
    tf32=True,                           # 3090 가속 옵션
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,3.143500
20,2.313200
30,1.551400
40,1.182800
50,1.091400
60,1.035600
70,0.957200
80,0.981600
90,0.928400
100,0.748700


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=750, training_loss=0.5807505915959676, metrics={'train_runtime': 15035.6704, 'train_samples_per_second': 0.798, 'train_steps_per_second': 0.05, 'total_flos': 5.5641636864e+17, 'train_loss': 0.5807505915959676, 'epoch': 3.0})

In [11]:
# 8. 모델 저장 및 업로드
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./kanana-customer-analysis/tokenizer_config.json',
 './kanana-customer-analysis/special_tokens_map.json',
 './kanana-customer-analysis/tokenizer.json')

In [12]:
# Hub 업로드
trainer.model.push_to_hub("ansui/kanana-customer-analysis")
tokenizer.push_to_hub("ansui/kanana-customer-analysis")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/ansui/kanana-customer-analysis/commit/1e61d594589445724a642a2c16c5057b0ad08305', commit_message='Upload tokenizer', commit_description='', oid='1e61d594589445724a642a2c16c5057b0ad08305', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ansui/kanana-customer-analysis', endpoint='https://huggingface.co', repo_type='model', repo_id='ansui/kanana-customer-analysis'), pr_revision=None, pr_num=None)

In [13]:
from peft import PeftModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. 경로 설정
adapter_path = output_dir
merged_repo_id = "ansui/kanana-customer-analysis-merged" # 새로 만들 레포 이름

print("Merging model... This requires ~16GB VRAM")

# 2. 베이스 모델을 16비트로 로드 (3090의 24GB VRAM 활용)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cpu", # 메모리 부족 방지를 위해 일단 CPU로 로드
    trust_remote_code=True
)

# 3. 토크나이저 로드 및 저장 준비
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 4. 어댑터 연결
model = PeftModel.from_pretrained(base_model, adapter_path)

# 5. 병합 실행 (가중치 합치기)
merged_model = model.merge_and_unload()

# 6. 로컬에 임시 저장 (검증용)
temp_save_path = "./merged_kanana_final"
merged_model.save_pretrained(temp_save_path)
tokenizer.save_pretrained(temp_save_path)

# 7. 허깅페이스 허브에 '통째로' 업로드
print("Pushing merged model to Hub...")
merged_model.push_to_hub(merged_repo_id, safe_serialization=True)
tokenizer.push_to_hub(merged_repo_id)

print(f"✅ 완료! 이제 '{merged_repo_id}' 레포지토리 하나만 있으면 어디서든 바로 쓸 수 있습니다.")

Merging model... This requires ~16GB VRAM


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Pushing merged model to Hub...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

✅ 완료! 이제 'ansui/kanana-customer-analysis-merged' 레포지토리 하나만 있으면 어디서든 바로 쓸 수 있습니다.


In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 1. 모델 설정
model_id = "ansui/kanana-customer-analysis-merged"

# 2. 4-bit 양자화 설정 (추론 속도 향상 및 메모리 절약)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 3. 토크나이저 및 모델 로드
print(f"Loading merged model from {model_id}...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="sdpa" # RTX 3090 가속
)
model.eval()

# 4. 테스트 함수 정의 (Kanana/Llama-3 포맷 적용)
def classify_customer(script):
    # Kanana/Llama-3 전용 인스트럭트 템플릿
    prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{SYSTEM_PROMPT}<|eot_id|>"
    prompt += f"<|start_header_id|>user<|end_header_id|>\n\n{script}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)
    
    # token_type_ids 에러 방지
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]
        
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10, 
            do_sample=False,    
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
    
    # 생성된 텍스트만 디코딩 (입력 프롬프트 부분 제외)
    result = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()
    return result

# 5. 실제 테스트 실행
testset = "아니 진짜 왜 이렇게 안 돼요? 아까부터 계속 기다리고 있는데 상담원 연결도 안 되고 짜증 나 죽겠네 진짜!",
prediction = classify_customer(testset)
print(f"분류 결과: {prediction}\n")

Loading merged model from ansui/kanana-customer-analysis-merged...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

분류 결과: S3

